### Versuch 2: Kalibrierung von Digitalkameras
##### In diesem Versuch werden die Eigenschaften von digitalen Kameras untersucht. Wir führen
##### dazu eine Kalibrierung des Kamerasensors durch, wie sie etwa bei industriellen Inspektionsanlagen,
##### bei der Fernerkundung durch Satelliten oder in der Astronomie gemacht wird.

#### Aufnahme eines Dunkelbildes
##### Um den pixelweisen Offset zu eliminieren, der durch thermisches Rauschen und Dunkelstrom verursacht wird.
#### Aufnahme eines Weißbildes
##### Um die unterschiedlichen Sensitivitäten der, einzelnen Pixel und die Vignettierung zu kompensieren.

In [93]:
import cv2
import numpy as np
from tabulate import tabulate

In [94]:
path_dark = "./Bilder/Dunkelbilder/2_"
path_white = "./Bilder/Weissbilder/3_"
path_gwk = "./Bilder/Grauwertkeil/1_"

In [95]:
"""
image_path: Der Pfad zum Bild, das verarbeitet werden soll
step: Die Schrittweite, mit der das Bild gescannt wird
threshold: Der Schwellenwert, der bestimmt, wann ein Schnitt gemacht wird (niedriger = sensibler)
"""

img_arr = []
def slice_image(image_path, step=20, threshold=10):
    image_array = []

    # Einlesen Grauwertkeil als Grauwertbild
    img_read = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_array.append(img_read)

    # Mittelwerte für die einzelnen Pixel
    img = np.mean(image_array, axis=0, dtype=float)

    subarrays = []
    slicingindex = []

    # Schleife für die einzelnen Bilder der Graustufen
    for n in range(0, len(img[0]) - step, step):
        # Wenn der Unterschied zwischen zwei Pixeln größer als der Schwellenwert ist, wird ein Schnitt gemacht
        if abs(int(img[0, n + step]) - int(img[0, n])) > threshold:
            slicingindex.append(n)
            # Ausschnitt 1-4 hinzufügen
            subarrays.append(img[:, n - 99:n])

    # letzten Ausschnitt hinzufügen
    subarrays.append(img[:, len(img[0]) - 99:len(img[0]) - 1])

    return subarrays


In [96]:
# Funktion zum Entfernen von Rauschen
def fix_noise(dark_img, white_img, input_img):
    # Dunkelbild von Eingabebild abziehen
    input_img = input_img - dark_img
    # Weißbild normieren, sodass sein Mittelwert 1 ist
    white_img = (white_img - dark_img) / np.mean(white_img)
    # Eingabebild durch normiertes Weißbild teilen
    input_img = input_img / white_img
    # korrigiertes Bild speichern
    cv2.imwrite("./Bilder/Grauwertkeil/Gwk_korrigiert.png", input_img)
    return input_img

In [97]:
sub_arrays = slice_image("./Bilder/Grauwertkeil/1_0.png")

res_mean = []
res_std = []
i = 0

table = []
for i, x in enumerate(sub_arrays):
    if x.size > 0:
        mean = np.round(np.mean(x), 2)
        std = np.round(np.std(x), 2)
    else:
        mean = np.nan
        std = np.nan

    res_mean.append(mean)
    res_std.append(std)

    table.append([i, mean, std])

In [98]:
print(tabulate(table, headers=["Index", "Mean", "Std"],
               tablefmt="simple", colalign=("left", "left", "left")))

Index    Mean    Std
-------  ------  -----
0        37.8    6.53
1        81.14   7.37
2        128.31  6.83
3        176.57  6.57
4        227.62  6.33


In [99]:
# Routine zum Einlesen der Bilder als Grauwertbilder
def get_image(image_path, number_images):
    # Array für die Bilder
    image_arr = []

    for j in range(number_images):
        img_read = cv2.imread(image_path + str(j) + ".png", cv2.IMREAD_GRAYSCALE)
        image_arr.append(img_read)

    # pixelweisen Mittelwert berechnen (double)
    return np.mean(image_arr, axis=0).astype(np.double)

In [100]:
# Einlesen der Bilder
dark_image = get_image(path_dark, 10)
white_image = get_image(path_white, 10)
input_image = get_image(path_gwk, 1)

# Rauschunterdrückung
res = fix_noise(dark_image, white_image, input_image)
cv2.imwrite("./Bilder/BilderNeu/Gwk_korrigiert.png", res.astype(np.uint8))

# Kontrast zu maximieren
dunkel_angepasst = dark_image * 50
weiss_angepasst = white_image * 50

cv2.imwrite("./Bilder/BilderNeu/dunkel_angepasst.png", dunkel_angepasst)
cv2.imwrite("./Bilder/BilderNeu/weiss_angepasst.png", weiss_angepasst)
cv2.imwrite("./Bilder/BilderNeu/dunkel_original.png", dark_image)
cv2.imwrite("./Bilder/BilderNeu/weiss_original.png", white_image)

True

In [101]:
# Überprüfen auf Hot pixel
def get_hot_pixels(dark_img):
    hot_pixels = dark_img > 0
    return hot_pixels

# Überprüfen auf Stuck pixel
def get_stuck_pixels(dark_img, weiss_img):
    stuck_pixels = dark_img == weiss_img
    return stuck_pixels

# Überprüfen auf Dead pixel
def get_dead_pixels(weiss_img):
    dead_pixels = weiss_img == 0
    return dead_pixels

# Markieren der Pixel im Bild
def mark_pixels(image, pixels, color):
    if len(image.shape) == 2: 
        marked_image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    else:
        marked_image = image
    marked_image[pixels] = color
    return marked_image

# Einlesen der Bilder
dark_image = cv2.imread("./Bilder/BilderNeu/dunkel_angepasst.png", cv2.IMREAD_GRAYSCALE)
white_image = cv2.imread("./Bilder/BilderNeu/weiss_angepasst.png", cv2.IMREAD_GRAYSCALE)

# Überprüfen und Markieren der Pixel
hot_pixels = get_hot_pixels(dark_image)
stuck_pixels = get_stuck_pixels(dark_image, white_image)
dead_pixels = get_dead_pixels(white_image)

marked_dark_image = mark_pixels(dark_image, hot_pixels, (0, 0, 255))  # Red for hot pixels
marked_dark_image = mark_pixels(marked_dark_image, stuck_pixels, (0, 255, 0))  # Green for stuck pixels
marked_white_image = mark_pixels(white_image, dead_pixels, (255, 0, 0))  # Blue for dead pixels

# Speichern der markierten Bilder
cv2.imwrite("./Bilder/BilderNeu/dark_image_marked.png", marked_dark_image)
cv2.imwrite("./Bilder/BilderNeu/white_image_marked.png", marked_white_image)

True

In [102]:
sub_arrays_fixed = slice_image("./Bilder/Grauwertkeil/Gwk_korrigiert.png", 15, 35)

res_mean_fixed = []
res_std_fixed = []
j = 0

table_fixed = []
for j, x in enumerate(sub_arrays_fixed):
    
    if x.size > 0:
        mean = np.round(np.mean(x), 2)
        std = np.round(np.std(x), 2)
    else:
        mean = np.nan
        std = np.nan

    res_mean_fixed.append(mean)
    res_std_fixed.append(std)

    table_fixed.append([j, mean, std])

print(tabulate(table_fixed, headers=["Index", "Mean", "Std"],
               tablefmt="simple", colalign=("left", "left", "left")))

Index    Mean    Std
-------  ------  -----
0        40.22   6.48
1        81.79   5.3
2        124.6   3.5
3        171.69  3.57
4        226.29  3.91


Kontrast maximierte Bilder:

<img src="./Bilder/BilderNeu/dunkel_angepasst.png" alt="dunkel kontrastmaximiert" style="width: 45%; display: inline-block;"/>
<img src="./Bilder/BilderNeu/weiss_angepasst.png" alt="weiß kontrastmaximiert" style="width: 45%; display: inline-block;"/>

Markierte Bilder: <br>
rot für Hotpixels (dark image) <br>
grün für Stuckpixels   (dark image) <br>
blau für Deadpixels   (white image)

<img src="./Bilder/BilderNeu/dark_image_marked.png" alt="Gwk korrigiert" style="width: 45%; display: inline-block;"/>
<img src="./Bilder/BilderNeu/white_image_marked.png" alt="Gwk korrigiert" style="width: 45%; display: inline-block;"/>

Originale Bilder:

<img src="./Bilder/BilderNeu/dunkel_original.png" alt="dunkel original" style="width: 45%; display: inline-block;"/>
<img src="./Bilder/BilderNeu/weiss_original.png" alt="weiß original" style="width: 45%; display: inline-block;"/>

Korrigierter Grauwertkeil (links) und Original Grauwertkeil (rechts):

<img src="./Bilder/Grauwertkeil/Gwk_korrigiert.png" alt="Gwk korrigiert" style="width: 45%; display: inline-block;"/>
<img src="./Bilder/Grauwertkeil/1_0.png" alt="Gwk original" style="width: 45%; display: inline-block;"/>